In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from gensim.models import Word2Vec
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import time
import nltk, re, string
from sklearn.model_selection import train_test_split, StratifiedKFold
import gc
from keras.layers import *
from keras.models import *
from keras.initializers import *
from keras.optimizers import *
import keras.backend as K
from keras.callbacks import *
from sklearn import metrics

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


['embeddings', 'train.csv', 'sample_submission.csv', 'test.csv']


In [2]:
max_features = 95000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 70 # max number of words in a question to use

In [3]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]
def clean_text(x):
    x = str(x)
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x

mispell_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", 
                "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", 
                "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", 
                "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", 
                "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", 
                "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have",
                "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not",
                "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": 
                "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", 
                "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", 
                "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", 
                "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", 
                "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", 
                "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", 
                "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", 
                "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", 
                "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", 
                "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization'}

def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re

mispellings, mispellings_re = _get_mispell(mispell_dict)
def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]
    return mispellings_re.sub(replace, text)

In [4]:
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')

train_df["question_text"] = train_df["question_text"].str.lower()
test_df["question_text"] = test_df["question_text"].str.lower()

train_df["question_text"] = train_df["question_text"].apply(lambda x: clean_text(x))
test_df["question_text"] = test_df["question_text"].apply(lambda x: clean_text(x))

train_df["question_text"] = train_df["question_text"].apply(lambda x: replace_typical_misspell(x))
test_df["question_text"] = test_df["question_text"].apply(lambda x: replace_typical_misspell(x))

train_X = train_df["question_text"].fillna("_##_").values
test_X = test_df["question_text"].fillna("_##_").values

train_y = train_df['target'].values


In [5]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_X)+list(test_X))
train_X = tokenizer.texts_to_sequences(train_X)
test_X = tokenizer.texts_to_sequences(test_X)

train_X = pad_sequences(train_X, maxlen=maxlen)
test_X = pad_sequences(test_X, maxlen=maxlen)

word_index = tokenizer.word_index

In [6]:
count = 0
nb_words = len(word_index)  # 200593
print(nb_words)

200593


In [7]:
start =time.clock()
all_data=pd.concat([train_df["question_text"],test_df["question_text"]])
# file_name = './word2vec.model'
# if not os.path.exists(file_name):
model = Word2Vec([[word for word in sentence.split(' ')] for sentence in all_data.values],
                     size=200, window=5, iter=10, workers=11, seed=2018, min_count=2)
#     model.save(file_name)
# else:
#     model = Word2Vec.load(file_name)
print("add word2vec finished....")  
end = time.clock()
print('Running time: %s Seconds'%(end-start))

add word2vec finished....
Running time: 750.5333810000001 Seconds


In [8]:
req_count = 10
for key in model.wv.similar_by_word('tony', topn =100):
    if len(key[0])==3:
        req_count -= 1
        print(key[0], key[1])
        if req_count == 0:
            break;

ned 0.7773456573486328
rey 0.6560162305831909
sam 0.6434968709945679
roy 0.6403218507766724
jon 0.6266370415687561
lee 0.619312047958374


/opt/conda/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [9]:
del train_df; gc.collect()
del test_df; gc.collect()
del all_data; gc.collect()
del tokenizer; gc.collect()

0

In [10]:
nb_words = min(max_features, len(word_index))
embedding_word2vec_matrix = np.zeros((nb_words, 200))  # (95000, 200)
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = model[word] if word in model else None
    if embedding_vector is not None:
        count += 1
        embedding_word2vec_matrix[i] = embedding_vector
    else:
        unk_vec = np.random.random(200) * 0.5
        unk_vec = unk_vec - unk_vec.mean()
        embedding_word2vec_matrix[i] = unk_vec

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  """
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [11]:
print(embedding_word2vec_matrix.shape)
# del model; gc.collect()  # biyao

(95000, 200)


In [12]:
EMBEDDING_FILE = '../input/embeddings/glove.840B.300d/glove.840B.300d.txt'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = -0.005838499,0.48782197
embed_size = all_embs.shape[1]

# word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_glove_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_glove_matrix[i] = embedding_vector

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """


In [13]:
embedding_matrix=np.concatenate((embedding_word2vec_matrix,embedding_glove_matrix),axis=1)

In [14]:
del word_index; gc.collect()
del embeddings_index; gc.collect()
del all_embs; gc.collect() # biyao
del embedding_word2vec_matrix; gc.collect()
del embedding_glove_matrix; gc.collect()

0

In [15]:
def bi_gru_model(embeddings_weight):
    K.clear_session()
    content = Input(shape=(maxlen,))
    embedding = Embedding(
        name="word_embedding",
        input_dim=embeddings_weight.shape[0],
        weights=[embeddings_weight],
        output_dim=embeddings_weight.shape[1],
        trainable=False)
#     embedding = Embedding(max_features, 500, weights=[embeddings_weight], trainable=False)(content)
    x = SpatialDropout1D(0.2)(embedding(content))
    x = Bidirectional(CuDNNGRU(200, return_sequences=True))(x)
    x = Bidirectional(CuDNNGRU(200, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    print("conc==", conc.shape)
    x = Dropout(0.2)(Activation(activation="relu")(BatchNormalization()(Dense(1000)(conc))))
    print("x shape==", x.shape)
    x = Activation(activation="relu")(BatchNormalization()(Dense(500)(x)))

    output = Dense(1, activation="sigmoid")(x)

    model = Model(inputs=content, outputs=output)
#     model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [16]:
print(train_y.shape)

(1306122,)


In [17]:
def train_pred(model, epochs=2):
    kfold = StratifiedKFold(n_splits=3, random_state=10, shuffle=True)
    pred_val_y = np.zeros(train_y.shape, )
#     print("pred_val_y==", pred_val_y.shape)
#     index = []
    for i, (train_index, valid_index) in enumerate(kfold.split(train_X, train_y)):
#         index.append(valid_index)
#         print("index::", index)
        X_train, X_val, Y_train, Y_val = train_X[train_index], train_X[valid_index], train_y[train_index], train_y[valid_index]
        filepath="weights_best.h5"
        checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=2, 
                                     save_best_only=True, mode='min')
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.6, patience=1, min_lr=0.0005, verbose=2)
        earlystopping = EarlyStopping(monitor='val_loss', min_delta=0.0005, patience=2, verbose=2, mode='auto')
        callbacks = [earlystopping, checkpoint, reduce_lr]
        if i == 0:print(model.summary()) 
        model.fit(X_train, Y_train, batch_size=512, epochs=epochs, 
                  validation_data=(X_val, Y_val), verbose=1, callbacks=callbacks)
        model.load_weights(filepath)
        pred_val_y[valid_index] = np.squeeze(model.predict([X_val], batch_size=1024, verbose=1))
#         print("pred_val_y[valid_index]==", pred_val_y[valid_index].shape)  # (653061,)
#         print("pred_y==", pred_val_y.shape)  # (1306122,)
        
#         best_thresh = 0.5
#         best_score = 0.0
#         for thresh in np.arange(0.1, 0.501, 0.01):
#             thresh = np.round(thresh, 2)
#             score = metrics.f1_score(Y_val, (pred_val_y[valid_index] > thresh).astype(int))
#             if score > best_score:
#                 best_thresh = thresh
#                 best_score = score
#         print("Val F1 Score: {:.4f}".format(best_score))    
    pred_test_y = model.predict([test_X], batch_size=1024, verbose=0)
    del model; gc.collect()
    return pred_val_y, pred_test_y

In [18]:
outputs = []

In [19]:
pred_val_y, pred_test_y = train_pred(bi_gru_model(embedding_matrix), epochs = 3)
outputs.append([pred_val_y, pred_test_y, 'bi_gru_model'])

conc== (?, 800)
x shape== (?, 1000)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 70)           0                                            
__________________________________________________________________________________________________
word_embedding (Embedding)      (None, 70, 500)      47500000    input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 70, 500)      0           word_embedding[0][0]             
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 70, 400)      842400      spatial_dropout1d_1[0][0]        
_________________________________________________________________________

In [20]:
for output in outputs:
    print(output[0], output[1])

[1.34684145e-02 1.73024796e-02 5.00112474e-05 ... 1.24474009e-03
 4.00871295e-06 6.26944820e-04] [[5.0600199e-04]
 [3.1848896e-05]
 [5.8393960e-04]
 ...
 [1.0794923e-03]
 [4.6406719e-03]
 [9.4433337e-01]]


In [21]:
# from sklearn.linear_model import LinearRegression
# X = np.asarray([outputs[i][0] for i in range(len(outputs))])
# X = X[...,0]
# reg = LinearRegression().fit(X.T, val_y)
# print(reg.score(X.T, val_y),reg.coef_)

In [22]:
val_y = train_y

In [23]:
# print(type(val_y[:10]))
# print(type(pred_val_y[:10]))
# print(type(outputs[0][:10]))

In [24]:
# pred_val_y = np.sum([outputs[i][0] * reg.coef_[i] for i in range(len(outputs))], axis = 0)
# pred_val_y = np.mean([outputs[i][0] for i in range(len(outputs))], axis = 0) # to avoid overfitting, just take average
pred_val_y = outputs[0][0]

thresholds = []
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    res = metrics.f1_score(val_y, (pred_val_y > thresh).astype(int))
    thresholds.append([thresh, res])
    print("F1 score at threshold {0} is {1}".format(thresh, res))
    
thresholds.sort(key=lambda x: x[1], reverse=True)
best_thresh = thresholds[0][0]
print("Best threshold: ", best_thresh)

F1 score at threshold 0.1 is 0.6238490585015654
F1 score at threshold 0.11 is 0.6330776999845935
F1 score at threshold 0.12 is 0.6417552100158025
F1 score at threshold 0.13 is 0.6493111723402364
F1 score at threshold 0.14 is 0.6565492680854524
F1 score at threshold 0.15 is 0.662695608278647
F1 score at threshold 0.16 is 0.6685082615561633
F1 score at threshold 0.17 is 0.6740012565484598
F1 score at threshold 0.18 is 0.6788047614261304
F1 score at threshold 0.19 is 0.6832512506301625
F1 score at threshold 0.2 is 0.6872859652049338
F1 score at threshold 0.21 is 0.6908441445687495
F1 score at threshold 0.22 is 0.6938445872397205
F1 score at threshold 0.23 is 0.6970270311398324
F1 score at threshold 0.24 is 0.6999912843307649
F1 score at threshold 0.25 is 0.7023505590280834
F1 score at threshold 0.26 is 0.7046595281914004
F1 score at threshold 0.27 is 0.7066016218955905
F1 score at threshold 0.28 is 0.7082689623576292
F1 score at threshold 0.29 is 0.7098258439045366
F1 score at threshold 0

In [25]:
# pred_test_y = np.sum([outputs[i][1] * reg.coef_[i] for i in range(len(outputs))], axis = 0)
# pred_test_y = np.mean([outputs[i][1] for i in range(len(outputs))], axis = 0)
pred_test_y = outputs[0][1]

pred_test_y = (pred_test_y > best_thresh).astype(int)
test_df = pd.read_csv("../input/test.csv", usecols=["qid"])
out_df = pd.DataFrame({"qid":test_df["qid"].values})
out_df['prediction'] = pred_test_y
out_df.to_csv("submission.csv", index=False)